![figure](lab4/politecnico_h-01.png)

# **Eletrónica Programável**
#### Licenciatura em Engenharia Eletrotécnica e de Computadores

## Trabalho Laboratorial 4 - Máquinas de Estados Finitos

-----------------------


## Introdução ##
Neste trabalho vamos implementar um conjunto de máquinas de estados usando uma descrição comportamental em VHDL, que permite implementar circuitos de média escala usando um nível de abstração superior. Este documento não fornece detalhes de utilização do Vivado. Sempre que tiver dúvidas a este nível, consulte o enunciado do Lab1.


### Objetivos ###

Neste trabalho popomos a implementação de duas máquinas de estados e um circuito de teste. Estas máquinas implementam circuitos essenciais em qualquer desenho síncrono, para além de servirem para os estudantes observarem a forma como podem descrever máquinas de estados em VHDL:
* Detector de flanco ascendente.
* Circuito de debouncing.
* Circuito de teste de flanco com debouncing.


Nas instruções abaixo **{sources}** refere-se a `C:\Xilinx\EEC_EP\sources` e **{labs}** refere-se a `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Passo 1 - Detector de flanco ascendente ##

### Passo 1.1 ### 

Um detector de flanco ascendente é um circuito que gera um pulso com a duração de um ciclo de relógio (podemos chamar-lhe um tick) sempre que detecta uma transição de '0' para '1', num determinado sinal de entrada. Este circuito pode ser desenhado como uma máquina de Moore ou de Mealy. Aqui, vamos analisar ambas as soluções para avaliar as suas diferenças.

Os diagramas de estados e de fluxo da máquina de Moore são apresentados na figura que se segue. Os estados zero e one indicam o nível lógico da entrada level, enquanto o estado edge indica que ocorreu uma transição ascendente, tal como mostra o diagrama temporal. 


<div>
<img src="lab4/fig1.png" width=900"/>
</div>

Se implementarmos o circuito como uma máquina de Mealy, passam a existir apenas dois estados, o estado *zero* e o estado *one*, tal como mostram os diagramas de estados e de fluxo da Figura abaixo. Quando estamos no estado *zero* e a entrada muda para '1', a saída **tick** é imediatamente asserida. Assim que a máquina estabiliza no estado *one*, a saída é novamente colocada a '0'. O diagrama temporal desta implementação é também apresentado na figura. 


<div>
<img src="lab4/fig2.png" width="800"/>
</div>


Note que esta máquina é mais compacta (menor número de estados) e responde mais rápido do que a máquina de Moore. A máquina de Mealy responde logo quando a transição é detectada, enquanto a de Moore só responde em t1. Como desvantagens, a máquina de Mealy é mais sensível a perturbações (glitches) no sinal de entrada e o pulso no sinal de saída não tem uma largura pré-definida. No entanto, se os circuitos a jusante forem síncronos com o flanco ascendente do mesmo sinal de relógio não há qualquer problema uma vez que o pulso na saída tick é estável em torno deste flanco. Desde que esta condição se verifique, a máquina de Mealy é geralmente a preferida para esta aplicação, devido à sua maior rapidez. Ainda nesta figura, podemos ver a corrspondente implementação em hardware (um flip-flop, uma porta AND e um inversor). 


* Comece por descarregar os arquivos fonte fornecidos (Github ou Moodle) e salve-os em `{sources}/lab4`.


* Abra o Vivado a partir do icon no ambinete de trabalho ou: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Selecione **Create Project** para começar o wizard. Na caixa de diálogo *Create A New Vivado Project*, selecione **Next**.


* Clique no botão *Procurar* do campo de localização do projeto do novo formulário de projeto, navegue até `c:/Xilinx/EEC_EP/labs` e clique em **Select**. Digite *lab4* no campo de nome do projeto. Certifique-se de que a caixa *Create Project Subdirectory* esteja marcada. Clique em **Next**.


* Selecione a opção **RTL Project** no formulário *Project Type* e clique em **Next**. Certifique-se de que a caixa *Do not specify sources at this time* está marcada.


* No formulário **Default Part**, selecione a opção *Boards* e escolha a placa **Basys3**. Clique em **Next** e depois em **Finish** para criar o projeto Vivado.


* No painel *Sources*, selecione **Add Sources** e siga os passos necessários (*Add or create design sources*) para adicionar o ficheiro **edge_detect.vhd**, que contém a descrição VHDL de diferentes arquiteturas do detector de flanco acima descrito. O ficheiro deve ser adicionado com a opção *Copy sources into project*.


* Abra o ficheiro no Vivado, observe o código. Analise as várias arquiteturas e se tiver alguma dúvida, chame o docente.



### Passo 1.2 ### 

Nesta seção, vamos verificar como as diferentes arquitecturas são implementadas pelo Vivado.

* Abra o ficheiro **edge_detect.vhd** e verifique que a arquitectura de Moore é a que está ativa (as outras duas estão comentadas.


* Escolha a operação *Open Elaborated Design* e abra o esquemático correspondente a este circuito. Repare que o esquemático inclui dois registos de estado, um conjunto de multiplexers que implementam a lógica de estado seguinte e um multiplexer que implementa a lógica de saída (lembre-se que os multiplexers podem ser usados para implementar circuitos lógicos).


* Feche a vista *Elaborated Design* na barra azul.


* Faça *Run Synthesis* e abra o projeto sintetizado. Abra o esquemático correspondente a este circuito e verifique as diferenças entre o circuito elaborado e um circuito sintetizado. Note que agora tem buffers de entrada (IBUF), um buffer global (BUFG) para o sinal de relógio, a lógica é mapeada em *Look-up-tables* (LUTs) e os registos são mapeados em flip-flops tipo D com *clear* e *enable* (FDCE).


* Observe a janela *Project Summary*. Verifique quantos recursos são usados nesta implementação. Deve ter 2 flip-flops, 2 LUTs, 4 buffers de I/O e um buffer global.


* Feche a vista de *Synthesized Design* na barra azul, vá ao ficheiro VHDL e ative a arquitetura em Máquina de Mealy (comente a arquitectura em Máquina de Moore). Faça **Save**.


* Repita os procedimentos anteriores para analisar a arquitectura em Máquina de Mealy. Note que agora deve ter um único registo de estado e que o número de recursos é mais reduzido.


* Feche a vista de *Synthesized Design* na barra azul e repita a alínea anterior para a arquitectura em *Gate Level*. Observe as diferenças ao nível do esquemático em RTL e pós-síntese. Em RTL aparecem exactamente os circuitos lógicos descritos em VHDL (e enão multiplexers genéricos), mas que são mapeados para LUTs e FDCE, da mesma forma. Os recursos usados nesta solução são os menores de todas as soluções analisadas nesta secção.


* Feche a vista de *Synthesized Design* na barra azul.


* Volte a ativar a arquitectura de Moore no ficheiro **edge_detect.vhd** e comente as restantes.

______

## Passo 2 - Circuito de debouncing ##

### Passo 2.1 ### 

Os interruptores e botões de pressão da placa de desenvolvimento são dispositivos mecânicos. Como tal, podem exibir trepidação (bounce) quando pressionados. O resultado desta trepidação é o aparecimento de pequenos pulsos indesejados (glitches), durante períodos de tempo que rondam tipicamente os 20ms. 

O objectivo de um circuito de debounce é eliminar estes glitches. Dependendo da sua implementação, podemos ter dois tipos de resposta, ilustrados na figura seguinte.


<div>
<img src="lab4/fig3.png" width="500"/>
</div>

Neste trabalho, vamos implementar o primeiro esquema usando uma Máquina de Estados e um gerador de relógio com Tclk=10ms. A máquina de estados deve ignorar os glitches e alterar o valor de saída apenas quando a entrada tiver estabilizado (entre 20ms e 30 ms). O diagrama de estados é representado na figura abaixo. Os estados *zero* e *one* indicam que o sinal de entrada estabilizou com o valor lógico '0' e '1', respectivamente. Os estados *waitx_1*, *waitx_2* e *waitx_3* garantem que a saída só é alterada quando a entrada estiver estável pelo menos durante 2 ciclos do contador (se o contador estiver perto do fim de contagem quando a máquina passa para estado *wait1_1*, o tempo de espera será próximo de 20ms, caso contrário será próximo de 30ms).


<div>
<img src="lab4/fig4.png" width="700"/>
</div>


* A descrição VHDL deste circuito é fornecida no ficheiro **db_fsm.vhd**. Adicione-o ao projeto Vivado (*Add or create design sources*), abra o ficheiro e observe o código.


* Analise o código e verifique que está de acordo com o diagrama de estados da figura acima. Note que o valor do genérico **N** é igual a vinte(N=20). Tente perceber porquê.


* Para simular, sintetizar ou implementar este módulo, deve primeiro pressionar o botão direito do rato sobre o módulo (na janela *Sources*), e depois escolher a opção **Set as Top** que aparece na janela de diálogo. Esta opção permite definir o módulo como sendo o circuito a considerar nas ações seguintes. 


* Adicione o ficheiro de estímulos **db_fsm_tb.vhd** e simule o seu funcionamento. Tente perceber porque motivo o valor do genérico **N** foi alterado na instanciação do módulo no *testbench*. Tente também perceber os tempos de espera entre alterações dos valores do estímulo de entada (**sw**).


* Adicione os sinais internos **state_reg** e **state_next** à janela de ondas do simulador. Para isso selecione o módulo UUT na janela Instances and Proces Name e arraste os sinais da janela Simulation Objects for UUT para a janela de ondas. 


* Observe os resultados da simulação e verifique se funciona como esperava.


* Feche a vista *Simulation* na faixa azul.



______________

## Passo 3 - Circuito de teste de flanco ascendente, com debouncing ##

### Passo 3.1 ### 

Para testar a eficácia dos circuitos implementados nas secções 1 e 2, vamos usar um circuito contador de trepidações (bounce counting). O diagrama de blocos é representado na figura abaixo. Note que a linha de relógio (net clk) não é totalmente representada para não complicar o diagrama. Por definição, as entradas de relógio dos módulos de um circuito síncrono estão ligadas ao sinal de relógio, que é único. Logo, não é necessário desenha-las (assume-se a ligação como óbvia). Desta forma o esquema fica mais simples, o que aumenta a sua clareza. 



<div>
<img src="lab4/fig5.png" width="900"/>
</div>


No ramo inferior, o sinal proveniente do botão de pressão passa pelo circuito de debouncing e pelo detector de flanco antes de atacar um contador de 8 bits. Assim, cada vez que o botão é pressionado, só deve ser gerado um pulso e o contador deve incrementar apenas uma unidade. No ramo superior, a falta do circuito de debounce deve fazer com que o contador conte também os glitches originados pela trepidação mecânica.


* Adicione o ficheiro **db_test.vhd** ao seu projeto, bem como o módulo **disp_hex_mux.vhd**, já utilizado em trabalhos anteriores. Note que o ficheiro *db_test* instancia o módulo de debounce e implemnta (em *Gate Level*) dois circuitos detectores de flanco. Analise o código VHDL. 


* Selecione o ficheiro **db_test.vhd** e garanta que é o ficheiro de topo na janela **Sources**.


* Siga os procedimentos habituais para sintetizar e implementar o circuito (processos Synthetize e Implement Design). Lembre-se que é necessário incluir o ficheiro de pinos Basys3, devidamente modificado, uma vez que vamos implementar este módulo na placa de desenvolvimento. 


* Verifique a funcionalidade do circuito de debouncing na Basys3.



_________

## Desafio ##


Usando o cronómetro digital do Lab3 e os circuitos aqui analisados, implemente um relógio digital (segundos e minutos) que permita definir o valor inicial de contagem de cada um dos algarismos, usando um botão de pressão para ajusar cada um deles. Cada vez que se pressiona um dos botões, o algarismo correspondente deve incrementar uma unidade. Este ajuste deve ser veito quando  relógio está no modo parado (**Go='0'**).

Deve primeiro desenhar o diagama de blocos do sistema e só depois passar para a sua implementação em VHDL.
